In [1]:
import hoomd

HOOMD-blue v2.1.1 DOUBLE HPMC_MIXED SSE SSE2 SSE3 SSE4_1 SSE4_2 AVX 
Compiled: 10/24/16
Copyright 2009-2016 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----


# Setting Initial Conditions

Before you can begin a simulation, you must specify the initial conditions of the particles and the periodic box dimensions. HOOMD provides a number of ways to set intial conditions. You can create particles on a lattice, read the configuration from a file, or specify data directly in python.

## Create a lattice

HOOMD can place particles on a lattice. For more information, see the [lattice](http://hoomd-blue.readthedocs.io/en/stable/module-hoomd-lattice.html#) and [init.create_lattice](http://hoomd-blue.readthedocs.io/en/stable/module-hoomd-init.html#hoomd.init.create_lattice) documentation.

HOOMD includes several standard lattices. Here is a square lattice in 2D:

In [2]:
hoomd.context.initialize('')
system = hoomd.init.create_lattice(unitcell=hoomd.lattice.sq(a=1.2), n=16)

HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 256 particles


Let's visualize the resulting system configuration with [Bokeh](http://bokeh.pydata.org/en/latest/)

In [3]:
from bokeh.plotting import figure, output_notebook, show
output_notebook();

Loading BokehJS ...

In [4]:
snap = system.take_snapshot()
p = figure(width=400, height=400)
p.circle(x=snap.particles.position[:,0], y=snap.particles.position[:,1], radius=0.5, 
         radius_units="data")
show(p);

And here is a hexagonal lattice:

In [5]:
hoomd.context.initialize('')
system = hoomd.init.create_lattice(unitcell=hoomd.lattice.hex(a=1.2), n=[16, 9])

notice(2): Group "all" created containing 288 particles


In [6]:
snap = system.take_snapshot()
p = figure(width=400, height=400)
p.circle(x=snap.particles.position[:,0], y=snap.particles.position[:,1], radius=0.5, 
         radius_units="data")
show(p);

### Lattice defaults

By default, the lattice generator names all particles A, and gives them mass 1.0, charge 0, diameter 1, moment of inertia [0,0,0], and orientation [1,0,0,0]. You can change the type name of the basic lattices in the argument list. Here is an example of a rectangular unit cell with two particle types of different sizes. The generic ``unitcell`` command takes in arbitrary unitcell vectors, along with positions, type names, mass, charge, etc... for every particle in the unit cell.

In [7]:
hoomd.context.initialize()
uc = hoomd.lattice.unitcell(N=2,
                            a1=[1, 0, 0],
                            a2=[0, 2, 0],
                            a3=[0, 0, 1],
                            dimensions=2,
                            position=[[0,0,0], [0.5, 1, 0]],
                            type_name=['A', 'B'],
                            diameter=[1.0, 0.5])
system = hoomd.init.create_lattice(unitcell=uc, n=[16, 8])

notice(2): Group "all" created containing 256 particles


In [8]:
snap = system.take_snapshot()
p = figure(width=400, height=400)
p.circle(x=snap.particles.position[:,0], y=snap.particles.position[:,1], 
         radius=snap.particles.diameter/2, radius_units="data")
show(p);

### A lattice of molecules

You can use the `unitcell` command to help build a lattice of molecules. As of hoomd v2.0, `unitcell` does not allow topology specification. But you can add bond topology manually using the snapshot.

In [9]:
hoomd.context.initialize()
uc = hoomd.lattice.unitcell(N=2,
                            a1=[2, 0, 0],
                            a2=[0, 2, 0],
                            a3=[0, 0, 1],
                            dimensions=2,
                            position=[[0,0,0], [0.6, 0.6, 0]],
                            type_name=['A', 'B'])

# Get a snapshot from the unitcell and add in bond topology
snap = uc.get_snapshot()
snap.bonds.resize(1)
snap.bonds.group[0] = [0, 1]
snap.bonds.types = ['bondA']

# replicate the lattice and initialize hoomd
snap.replicate(4,4,1)
system = hoomd.init.read_snapshot(snap)

notice(2): Group "all" created containing 32 particles


In [10]:
snap = system.take_snapshot()
p = figure(width=400, height=400)
p.circle(x=snap.particles.position[:,0], y=snap.particles.position[:,1], radius=0.5, 
         radius_units="data")
show(p);